In [1]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [88]:
import os
import glob
import pandas as pd
import yaml

# 🔹 Change this to your folder containing YAML files
data_folder = r"data"  # <-- REPLACE THIS with your folder path

# Function to flatten a single match YAML file into rows
def flatten_match_yaml(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        match = yaml.safe_load(f)

    rows = []
    info = match.get("info", {})
    match_id = os.path.splitext(os.path.basename(file_path))[0]

    # Match-level info
    date = info.get("dates", [None])[0]
    city = info.get("city")
    venue = info.get("venue")
    teams = info.get("teams", [None, None])
    toss_winner = info.get("toss", {}).get("winner")
    toss_decision = info.get("toss", {}).get("decision")
    outcome = info.get("outcome", {})
    winner = outcome.get("winner")
    by = outcome.get("by", {})
    result_by = f"{list(by.values())[0]} {list(by.keys())[0]}" if by else None

    # Iterate over innings
    for inn in match.get("innings", []):
        for label, inn_data in inn.items():
            batting = inn_data.get("team")
            bowling = teams[1] if batting == teams[0] else teams[0]

            # Iterate over deliveries
            for delivery in inn_data.get("deliveries", []):
                ball_key, ball_data = list(delivery.items())[0]
                over, ball_no = map(int, str(ball_key).split("."))

                runs = ball_data.get("runs", {})
                extras_detail = ball_data.get("extras", {})
                wicket = ball_data.get("wicket", {})

                rows.append({
                    "match_id": match_id,
                    "date": date,
                    "city": city,
                    "venue": venue,
                    "team1": teams[0],
                    "team2": teams[1],
                    "toss_winner": toss_winner,
                    "toss_decision": toss_decision,
                    "winner": winner,
                    "result_by": result_by,
                    "innings_label": label,
                    "batting_team": batting,
                    "bowling_team": bowling,
                    "over": over,
                    "ball": ball_no,
                    "batsman": ball_data.get("batsman"),
                    "non_striker": ball_data.get("non_striker"),
                    "bowler": ball_data.get("bowler"),
                    "runs_batsman": runs.get("batsman", 0),
                    "runs_extras": runs.get("extras", 0),
                    "runs_total": runs.get("total", 0),
                    "extra_types": ";".join(extras_detail.keys()) if extras_detail else None,
                    "wicket_kind": wicket.get("kind"),
                    "wicket_player_out": wicket.get("player_out"),
                    "wicket_fielders": ";".join(wicket.get("fielders", [])) if wicket.get("fielders") else None
                })
    return rows

# Function to load all YAML files from folder
def load_all_matches(data_folder):
    files = glob.glob(os.path.join(data_folder, "*.yaml")) + \
            glob.glob(os.path.join(data_folder, "*.yml"))

    all_rows = []
    for i, file in enumerate(files, start=1):
        try:
            all_rows.extend(flatten_match_yaml(file))
            if i % 100 == 0:
                print(f"Processed {i} files...")
        except Exception as e:
            print(f"Skipping {file} due to error: {e}")
    return pd.DataFrame(all_rows)

# 🔹 Run the loader
df = load_all_matches(data_folder)

print(f"✅ Loaded {len(df)} deliveries from {df['match_id'].nunique()} matches.")
df.head()


Processed 100 files...
Processed 200 files...
Processed 300 files...
Processed 400 files...
Processed 500 files...
Processed 600 files...
Processed 700 files...
Processed 800 files...
Processed 900 files...
Processed 1000 files...
Processed 1100 files...
Processed 1200 files...
Processed 1300 files...
Processed 1400 files...
Processed 1500 files...
Processed 1600 files...
Processed 1700 files...
Processed 1800 files...
Processed 1900 files...
Processed 2000 files...
Skipping data\1383095.yaml due to error: 'list' object has no attribute 'get'
Processed 2100 files...
Processed 2200 files...
Processed 2300 files...
Processed 2400 files...
Processed 2500 files...
Processed 2600 files...
Processed 2700 files...
Processed 2800 files...
Processed 2900 files...
Processed 3000 files...
Processed 3100 files...
Processed 3200 files...
Skipping data\1459387.yaml due to error: 'list' object has no attribute 'get'
Processed 3300 files...
Processed 3400 files...
Processed 3500 files...
Skipping data

,match_id,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_by,...,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,extra_types,wicket_kind,wicket_player_out,wicket_fielders
0,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,None,None,None,None
1,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,None,None,None,None
2,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,1,0,1,None,None,None,None
3,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,2,0,2,None,None,None,None
4,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,0,0,0,None,None,None,None


In [89]:
df

,match_id,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_by,...,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,extra_types,wicket_kind,wicket_player_out,wicket_fielders
0,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,None,None,None,None
1,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,None,None,None,None
2,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,1,0,1,None,None,None,None
3,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,2,0,2,None,None,None,None
4,1001349,2017-02-17,None,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,0,0,0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953887,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,TM Head,PM Nevill,SS Pathirana,1,0,1,None,None,None,None
953888,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,PM Nevill,TM Head,SS Pathirana,3,0,3,None,None,None,None
953889,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,TM Head,PM Nevill,SS Pathirana,0,0,0,None,None,None,None
953890,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,TM Head,PM Nevill,SS Pathirana,0,0,0,None,None,None,None


In [90]:
df.columns

Index(['match_id', 'date', 'city', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result_by', 'innings_label', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'runs_batsman', 'runs_extras', 'runs_total', 'extra_types',
       'wicket_kind', 'wicket_player_out', 'wicket_fielders'],
      dtype='object')

In [91]:
# Save DataFrame as CSV in the current folder
df.to_csv("t20_deliveries.csv", index=False, encoding="utf-8")
print("💾 CSV saved as t20_deliveries.csv")


💾 CSV saved as t20_deliveries.csv


In [92]:

# Load CSV into a DataFrame
df2 = pd.read_csv("t20_deliveries.csv")

# Check first few rows
print(df.shape)
df2.head()


(953892, 25)


,match_id,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_by,...,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,extra_types,wicket_kind,wicket_player_out,wicket_fielders
0,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,NaN,NaN
1,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,NaN,NaN
2,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,1,0,1,NaN,NaN,NaN,NaN
3,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,2,0,2,NaN,NaN,NaN,NaN
4,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,0,0,0,NaN,NaN,NaN,NaN


In [93]:
df2[['over','ball']]

,over,ball
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
953887,17,1
953888,17,2
953889,17,3
953890,17,4


In [95]:
df2.columns

Index(['match_id', 'date', 'city', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result_by', 'innings_label', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'runs_batsman', 'runs_extras', 'runs_total', 'extra_types',
       'wicket_kind', 'wicket_player_out', 'wicket_fielders'],
      dtype='object')

In [96]:
df2['wicket_player_out'] = df2['wicket_player_out'].fillna(0).astype(str)  # as string

In [97]:
df2.head()

,match_id,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_by,...,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,extra_types,wicket_kind,wicket_player_out,wicket_fielders
0,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,0,NaN
1,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,0,NaN
2,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,1,0,1,NaN,NaN,0,NaN
3,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,2,0,2,NaN,NaN,0,NaN
4,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,0,0,0,NaN,NaN,0,NaN


In [101]:
df2["wicket_player_out"].unique()

array(['0', 'M Klinger', 'AJ Finch', ..., 'Hafiz Qaleem', 'Farhan Ahmed',
       'Qadeer Ahmed'], dtype=object)

In [100]:
df2.columns

Index(['match_id', 'date', 'city', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result_by', 'innings_label', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'runs_batsman', 'runs_extras', 'runs_total', 'extra_types',
       'wicket_kind', 'wicket_player_out', 'wicket_fielders'],
      dtype='object')

In [116]:
df3=df2[df2['innings_label'] == '1st innings']

In [117]:
df3

,match_id,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_by,...,batsman,non_striker,bowler,runs_batsman,runs_extras,runs_total,extra_types,wicket_kind,wicket_player_out,wicket_fielders
0,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,0,NaN
1,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN,0,NaN
2,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,AJ Finch,M Klinger,SL Malinga,1,0,1,NaN,NaN,0,NaN
3,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,2,0,2,NaN,NaN,0,NaN
4,1001349,2017-02-17,NaN,Melbourne Cricket Ground,Australia,Sri Lanka,Sri Lanka,field,Sri Lanka,5 wickets,...,M Klinger,AJ Finch,SL Malinga,0,0,0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953777,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,SMSM Senanayake,DM de Silva,MA Starc,1,0,1,NaN,NaN,0,NaN
953778,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,DM de Silva,SMSM Senanayake,MA Starc,0,0,0,NaN,NaN,0,NaN
953779,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,DM de Silva,SMSM Senanayake,MA Starc,0,0,0,NaN,caught,DM de Silva,JP Faulkner
953780,995469,2016-09-09,Colombo,R Premadasa Stadium,Sri Lanka,Australia,Sri Lanka,bat,Australia,4 wickets,...,SMSM Senanayake,RAS Lakmal,MA Starc,2,0,2,NaN,NaN,0,NaN


In [118]:
delivery_df=df3[["match_id","bowling_team","batting_team","over","ball","batsman","bowler","runs_batsman","wicket_player_out","city","venue"]]

In [119]:
delivery_df

,match_id,bowling_team,batting_team,over,ball,batsman,bowler,runs_batsman,wicket_player_out,city,venue
0,1001349,Sri Lanka,Australia,0,1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
1,1001349,Sri Lanka,Australia,0,2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
2,1001349,Sri Lanka,Australia,0,3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground
3,1001349,Sri Lanka,Australia,0,4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground
4,1001349,Sri Lanka,Australia,0,5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...,...,...,...
953777,995469,Australia,Sri Lanka,19,3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium
953778,995469,Australia,Sri Lanka,19,4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium
953779,995469,Australia,Sri Lanka,19,5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium
953780,995469,Australia,Sri Lanka,19,6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium


In [120]:
delivery_df['batting_team'].unique()

array(['Australia', 'Hong Kong', 'Zimbabwe', 'India', 'Bangladesh',
       'New Zealand', 'South Africa', 'England', 'West Indies',
       'Sri Lanka', 'Pakistan', 'Scotland', 'Oman', 'Ireland',
       'Papua New Guinea', 'United Arab Emirates', 'Netherlands',
       'Thailand', 'Uganda', 'Malaysia', 'Botswana', 'Malawi',
       'Sierra Leone', 'Mozambique', 'Namibia', 'Lesotho', 'Nepal',
       'China', 'Kuwait', 'Vanuatu', 'Philippines',
       'United States of America', 'Germany', 'Nigeria', 'Tanzania',
       'Japan', 'Indonesia', 'Fiji', 'Samoa', 'Ghana', 'Kenya',
       'Guernsey', 'Denmark', 'Jersey', 'Italy', 'Norway', 'Maldives',
       'Mali', 'Singapore', 'Bermuda', 'Canada', 'Cayman Islands',
       'Portugal', 'Gibraltar', 'Spain', 'Bhutan', 'Qatar', 'Iran',
       'Austria', 'Belgium', 'Isle of Man', 'Bulgaria', 'Romania',
       'Luxembourg', 'Czech Republic', 'Rwanda', 'Greece', 'Serbia',
       'Malta', 'France', 'Sweden', 'Finland', 'Eswatini', 'Cameroon',
       'Hu

In [121]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]

In [122]:
delivery_df=delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df=delivery_df[delivery_df['bowling_team'].isin(teams)]

In [124]:
delivery_df

,match_id,bowling_team,batting_team,over,ball,batsman,bowler,runs_batsman,wicket_player_out,city,venue
0,1001349,Sri Lanka,Australia,0,1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
1,1001349,Sri Lanka,Australia,0,2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground
2,1001349,Sri Lanka,Australia,0,3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground
3,1001349,Sri Lanka,Australia,0,4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground
4,1001349,Sri Lanka,Australia,0,5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...,...,...,...
953777,995469,Australia,Sri Lanka,19,3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium
953778,995469,Australia,Sri Lanka,19,4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium
953779,995469,Australia,Sri Lanka,19,5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium
953780,995469,Australia,Sri Lanka,19,6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium


In [126]:
delivery_df['over_ball'] = delivery_df['over'].astype(str) + '.' + delivery_df['ball'].astype(str)
delivery_df['over_ball'] = delivery_df['over_ball'].astype(float)

In [127]:
delivery_df

,match_id,bowling_team,batting_team,over,ball,batsman,bowler,runs_batsman,wicket_player_out,city,venue,over_ball
0,1001349,Sri Lanka,Australia,0,1,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,0.1
1,1001349,Sri Lanka,Australia,0,2,AJ Finch,SL Malinga,0,0,NaN,Melbourne Cricket Ground,0.2
2,1001349,Sri Lanka,Australia,0,3,AJ Finch,SL Malinga,1,0,NaN,Melbourne Cricket Ground,0.3
3,1001349,Sri Lanka,Australia,0,4,M Klinger,SL Malinga,2,0,NaN,Melbourne Cricket Ground,0.4
4,1001349,Sri Lanka,Australia,0,5,M Klinger,SL Malinga,0,0,NaN,Melbourne Cricket Ground,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
953777,995469,Australia,Sri Lanka,19,3,SMSM Senanayake,MA Starc,1,0,Colombo,R Premadasa Stadium,19.3
953778,995469,Australia,Sri Lanka,19,4,DM de Silva,MA Starc,0,0,Colombo,R Premadasa Stadium,19.4
953779,995469,Australia,Sri Lanka,19,5,DM de Silva,MA Starc,0,DM de Silva,Colombo,R Premadasa Stadium,19.5
953780,995469,Australia,Sri Lanka,19,6,SMSM Senanayake,MA Starc,2,0,Colombo,R Premadasa Stadium,19.6


In [128]:
delivery_df=delivery_df[['match_id','bowling_team','batting_team','over','ball','runs_batsman','wicket_player_out','city','venue','over_ball']]

In [129]:
delivery_df

,match_id,bowling_team,batting_team,over,ball,runs_batsman,wicket_player_out,city,venue,over_ball
0,1001349,Sri Lanka,Australia,0,1,0,0,NaN,Melbourne Cricket Ground,0.1
1,1001349,Sri Lanka,Australia,0,2,0,0,NaN,Melbourne Cricket Ground,0.2
2,1001349,Sri Lanka,Australia,0,3,1,0,NaN,Melbourne Cricket Ground,0.3
3,1001349,Sri Lanka,Australia,0,4,2,0,NaN,Melbourne Cricket Ground,0.4
4,1001349,Sri Lanka,Australia,0,5,0,0,NaN,Melbourne Cricket Ground,0.5
...,...,...,...,...,...,...,...,...,...,...
953777,995469,Australia,Sri Lanka,19,3,1,0,Colombo,R Premadasa Stadium,19.3
953778,995469,Australia,Sri Lanka,19,4,0,0,Colombo,R Premadasa Stadium,19.4
953779,995469,Australia,Sri Lanka,19,5,0,DM de Silva,Colombo,R Premadasa Stadium,19.5
953780,995469,Australia,Sri Lanka,19,6,2,0,Colombo,R Premadasa Stadium,19.6


In [130]:
# Save DataFrame as CSV in the current folder
delivery_df.to_csv("extracted_data.csv", index=False, encoding="utf-8")
print("💾 CSV saved as extracted_data.csv")


💾 CSV saved as extracted_data.csv


In [131]:
# Load CSV into a DataFrame
df = pd.read_csv("extracted_data.csv")

# Check first few rows
print(df.shape)
df.head()


(164860, 10)


,match_id,bowling_team,batting_team,over,ball,runs_batsman,wicket_player_out,city,venue,over_ball
0,1001349,Sri Lanka,Australia,0,1,0,0,NaN,Melbourne Cricket Ground,0.1
1,1001349,Sri Lanka,Australia,0,2,0,0,NaN,Melbourne Cricket Ground,0.2
2,1001349,Sri Lanka,Australia,0,3,1,0,NaN,Melbourne Cricket Ground,0.3
3,1001349,Sri Lanka,Australia,0,4,2,0,NaN,Melbourne Cricket Ground,0.4
4,1001349,Sri Lanka,Australia,0,5,0,0,NaN,Melbourne Cricket Ground,0.5
